In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_json("/kaggle/input/sarcasm-detection-through-nlp/Sarcasm_Headlines_Dataset.json", lines = True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
df.to_csv('sarcasm_csv', index = False)
df = pd.read_csv("/kaggle/working/sarcasm_csv")
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
df.isnull().sum()

article_link    0
headline        0
is_sarcastic    0
dtype: int64

In [5]:
y = df.is_sarcastic
x = df.drop('is_sarcastic', axis = 1)

In [6]:
print(x.shape)
print(y.shape)

(26709, 2)
(26709,)


In [7]:
import tensorflow as tf

2024-04-25 17:54:17.180375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 17:54:17.180475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 17:54:17.321239: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
tf.__version__

'2.15.0'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import RMSprop

In [10]:
voca_size = 10000

In [11]:
text = x.copy()

In [12]:
text['headline'][1]

"the 'roseanne' revival catches up to our thorny political mood, for better and worse"

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(text)):
    review = re.sub('[^a-zA-Z]', ' ', text['headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus

['former versac store clerk sue secret black code minor shopper',
 'roseann reviv catch thorni polit mood better wors',
 'mom start fear son web seri closest thing grandchild',
 'boehner want wife listen come altern debt reduct idea',
 'j k rowl wish snape happi birthday magic way',
 'advanc world women',
 'fascin case eat lab grown meat',
 'ceo send kid school work compani',
 'top snake handler leav sink huckabe campaign',
 'friday morn email insid trump presser age',
 'airlin passeng tackl man rush cockpit bomb threat',
 'facebook reportedli work healthcar featur app',
 'north korea prais trump urg us voter reject dull hillari',
 'actual cnn jeffrey lord indefens',
 'barcelona hold huge protest support refuge',
 'nuclear bomb deton rehears spider man music',
 'cosbi lawyer ask accus come forward smear legal team year ago',
 'stock analyst confus frighten boar market',
 'bloomberg program build better citi got bigger',
 'craig hick indict',
 'courtroom sketch artist clear manga influe

In [17]:
onehot_repr=[one_hot(words,voca_size)for words in corpus]
onehot_repr

[[2134, 5613, 743, 6070, 816, 224, 7543, 1429, 5790, 3837],
 [2630, 4031, 1076, 6871, 6729, 7063, 7206, 3894],
 [9802, 3250, 7924, 2156, 2070, 1470, 8752, 2037, 493],
 [5971, 6001, 7500, 6784, 3702, 8582, 3146, 9523, 7193],
 [5025, 6527, 7329, 1574, 4842, 8998, 9768, 6105, 943],
 [4374, 5070, 5270],
 [8685, 6012, 3628, 8422, 8734, 2057],
 [9839, 1045, 3038, 5387, 2046, 9924],
 [5566, 3237, 6535, 2195, 120, 9534, 2744],
 [7562, 8558, 5687, 2472, 3163, 2431, 7201],
 [2744, 2198, 7844, 1072, 4792, 8213, 8869, 1433],
 [6324, 903, 2046, 1910, 7005, 9849],
 [703, 188, 6028, 3163, 2828, 9616, 1208, 4829, 4299, 8816],
 [2137, 3062, 3915, 4774, 3638],
 [6596, 2958, 8500, 7248, 3479, 8903],
 [7784, 8869, 9846, 4755, 3401, 1072, 2189],
 [8064, 525, 2874, 8177, 3702, 3588, 9585, 1398, 3000, 5860, 3406],
 [4897, 823, 2826, 5581, 1924, 7657],
 [2927, 7919, 5835, 7206, 7979, 6332, 9075],
 [3812, 9679, 2948],
 [2488, 7993, 5042, 3642, 9789, 6681],
 [3163, 9408, 256, 7751, 8713, 8144, 8905, 4925, 2382,

In [18]:
print(corpus[1])
print(onehot_repr[1])

roseann reviv catch thorni polit mood better wors
[2630, 4031, 1076, 6871, 6729, 7063, 7206, 3894]


In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2134 5613  743 ...    0    0    0]
 [2630 4031 1076 ...    0    0    0]
 [9802 3250 7924 ...    0    0    0]
 ...
 [7614 4798    0 ...    0    0    0]
 [8585  605 7996 ...    0    0    0]
 [8098 3627 9999 ...    0    0    0]]


In [20]:
print(embedded_docs[1])

[2630 4031 1076 6871 6729 7063 7206 3894    0    0    0    0    0    0
    0    0    0    0    0    0]


In [21]:
embedding_vector_features = 50  
model = Sequential()
model.add(Embedding(voca_size, embedding_vector_features))
model.add(Bidirectional(LSTM(128, return_sequences=True)))  
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(64, return_sequences=True)))  
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(32)))  
model.add(Dropout(0.2))  
model.add(Dense(1, activation='sigmoid'))

optimizer_rmsprop = RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer_rmsprop, metrics=['accuracy'])

In [22]:
len(embedded_docs),y.shape

(26709, (26709,))

In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((26709, 20), (26709,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=18,batch_size=64)

Epoch 1/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9556 - loss: 0.1364 - val_accuracy: 0.7550 - val_loss: 0.7044
Epoch 2/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9597 - loss: 0.1263 - val_accuracy: 0.7443 - val_loss: 0.7948
Epoch 3/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9649 - loss: 0.1136 - val_accuracy: 0.7519 - val_loss: 0.7637
Epoch 4/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9678 - loss: 0.1062 - val_accuracy: 0.7422 - val_loss: 0.8219
Epoch 5/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9730 - loss: 0.0878 - val_accuracy: 0.7465 - val_loss: 0.8557
Epoch 6/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9763 - loss: 0.0807 - val_accuracy: 0.7469 - val_loss: 0.9223
Epoch 7/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9807 - loss: 0.0671 - val_accuracy: 0.7404 - val_loss: 0.8951
Epoch 8/18
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9812 - loss: 0.0606 - val_accu

In [28]:
y_pred=model.predict(X_test)

276/276 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [29]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred)

array([[3766, 1138],
       [1244, 2666]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7297481279782164

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76      4904
           1       0.70      0.68      0.69      3910

    accuracy                           0.73      8814
   macro avg       0.73      0.72      0.73      8814
weighted avg       0.73      0.73      0.73      8814



# Now with transformer


In [34]:
!pip install transformers

In [35]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained("jkhan447/sarcasm-detection-Bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("jkhan447/sarcasm-detection-Bert-base-uncased", from_pt=True)


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequ

In [37]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=18, batch_size=64)

Epoch 1/18


I0000 00:00:1714068124.183653     111 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


280/280 [==============================] - 146s 327ms/step - loss: 7.0353 - accuracy: 0.5635 - val_loss: 7.1502 - val_accuracy: 0.5564
Epoch 2/18
280/280 [==============================] - 78s 277ms/step - loss: 7.0381 - accuracy: 0.5633 - val_loss: 7.1502 - val_accuracy: 0.5564
Epoch 3/18
280/280 [==============================] - 77s 276ms/step - loss: 7.0381 - accuracy: 0.5633 - val_loss: 7.1502 - val_accuracy: 0.5564
Epoch 4/18
280/280 [==============================] - 77s 276ms/step - loss: 7.0381 - accuracy: 0.5633 - val_loss: 7.1502 - val_accuracy: 0.5564
Epoch 5/18
280/280 [==============================] - 77s 276ms/step - loss: 7.0381 - accuracy: 0.5633 - val_loss: 7.1502 - val_accuracy: 0.5564
Epoch 6/18
280/280 [==============================] - 77s 276ms/step - loss: 7.0381 - accuracy: 0.5633 - val_loss: 7.1502 - val_accuracy: 0.5564
Epoch 7/18
280/280 [==============================] - 77s 276ms/step - loss: 7.0381 - accuracy: 0.5633 - val_loss: 7.1502 - val_accuracy: 0.

In [38]:
y_pred=model.predict(X_test)

276/276 [==============================] - 23s 48ms/step


In [55]:
threshold = 0.5
y_pred_binary = np.where(y_pred_thresholded > threshold, 1, 0)
y_pred_binary = y_pred_binary[:, 1] 
# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)

Accuracy: 0.5563875652371227


### My Model wins 